In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
! pip install datasets sentencepiece transformers evaluate rouge_score bert_score huggingface_hub sentencepiece
! pip3 install - U scikit-learn scipy matplotlib

In [3]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import torch.nn as nn
from transformers import pipeline
import pandas as pd

In [4]:
df_train = pd.read_csv("/content/drive/MyDrive/df_train.csv")
df_val = pd.read_csv("/content/drive/MyDrive/df_val.csv")

In [6]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "t5-small"

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [8]:
def tokenize_function(examples):
    inputs = [ex for ex in examples["instruction"]]
    targets = [ex for ex in examples["output"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, truncation=True, padding='max_length'
    )
    return model_inputs

In [ ]:
tokenized_train_datasets = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names,
)
tokenized_val_datasets = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=val_dataset.column_names,
)

In [ ]:
tokenized_train_datasets['labels']

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=8,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",  # Save the model at the end of each epoch
    save_total_limit=1,  # Limit the number of saved checkpoints to 3
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_val_datasets,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
# save the model
trainer._save("./t5-finetuned/model")
tokenizer.save_pretrained("./t5-finetuned/tokenizer")

('./t5-finetuned/tokenizer/tokenizer_config.json',
 './t5-finetuned/tokenizer/special_tokens_map.json',
 './t5-finetuned/tokenizer/spiece.model',
 './t5-finetuned/tokenizer/added_tokens.json')

In [ ]:
results = trainer.evaluate()

In [ ]:
results

{'eval_loss': 1.3074653148651123,
 'eval_runtime': 11.993,
 'eval_samples_per_second': 34.854,
 'eval_steps_per_second': 4.419,
 'epoch': 8.0}

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
model_name = "./t5-finetuned/model"
tokenizer = T5Tokenizer.from_pretrained(
    model_name)
model = T5ForConditionalGeneration.from_pretrained(
    model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def generate_response(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model.generate(input_ids,
                             min_length=5,
                             max_length=300,
                             do_sample=True, num_beams=5, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(
        outputs[0], skip_special_tokens=True)
    return generated_text


generate_response("how to be healthy")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("") # Push the model to the Hugging Face Hub

In [ ]:
tokenizer.push_to_hub("") # Push the tokenizer to the Hugging Face Hub